# [DE:Betretungsverbote für Gebiete im Winter](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_für_Gebiete_im_Winter)

* Extracts all ways via Overpass and filter/analyze further based on the definition above
* perform some basic analyis (multiple types, untyped)
* create a GeoJSON with feature properties set to type<1-8> for direct rendering
  * style properties are set for https://geojson.io

## ToDo:

* support for relations
* adjust style properties for types (within limits of both https://geojson.io and leaflet)
* visualize geojson in Notebook
* direct import for geojson.io via API?


## Overpass Basis-Query

In [ ]:
import sys, re, os
# https://github.com/jazzband/geojson
import geojson
# https://python-overpy.readthedocs.io/en/latest/index.html
import overpy
from IPython.core.display import display, HTML

silent = False
if sys.argv[1] == "silent":
    silent = True

api = overpy.Overpass()

# http://norbertrenner.de/osm/bbox.html
bbox = "47.378,11.078,47.768,13.111"
# [bbox:{bbox}];
query = f"""
way["boundary"="protected_area"]["protect_class"="14"];
(._;>;);
out body;
"""

result = api.query(query)
if not silent:
    print (f"retrieved {len(result.ways)} ways")

In [ ]:
# transform all tags into the names used in the "SQL-like" queries in the Wiki
def get_tags(tags):
    seasonal = way.tags.get("seasonal")
    ski = way.tags.get("ski")
    ski_conditional = way.tags.get("ski:conditional") or ""
    access = way.tags.get("access")
    access_conditional = way.tags.get("access:conditional") or ""
    access_offroad = way.tags.get("access:offroad") or ""
    access_offroad_conditional = way.tags.get("access:offroad:conditional") or ""
    
    return seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional

## 1 - [Ganzjähriges Befahrungsverbot für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hriges_Befahrungsverbot_f.C3.BCr_Skifahrer)


In [ ]:
allTypes= []
ways = []
for way in result.ways:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(way.tags)
    
    if (
        seasonal is None
        or seasonal == 'no'
    ) and (
        ski in ['no', 'private'] or (
            access in ['no','private'] and (
                ski is None
                or not ski in ['yes','designated','permissive']
            )
        )
    ):
        ways.append(way)
allTypes.append(ways)

## 2 - [Monatsabhängiges oder saisonales Befahrungsverbot für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngiges_oder_saisonales_Befahrungsverbot_f.C3.BCr_Skifahrer)

In [ ]:
ways = []
for way in result.ways:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(way.tags)

    if (
        (
            (
                not (seasonal is None or seasonal == 'no')
            ) and (
                ski in ['no', 'private']
                or re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
                or (
                    (
                        access in ['no','private']
                        or re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                    ) and (
                        ski is None
                        or not ski in ['yes','designated','permissive']
                    )
                )
            )
            ) or (
                re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
                or (
                    (
                        re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                    ) and (
                        ski is None
                    )
                )
            )
        ):
        ways.append(way)
allTypes.append(ways)

## 3 - [Ganzjährige "Bitte um Nichtbefahrung" für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hrige_.22Bitte_um_Nichtbefahrung.22_f.C3.BCr_Skifahrer)

In [ ]:
ways = []
for way in result.ways:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(way.tags)
    
    if ( 
        seasonal is None
        or seasonal == 'no'
    ) and (
        ski == 'discouraged'
        or (
            access == 'discouraged' and (
                ski is None
                or not ski in ['yes','designated','permissive']
            )
        )
    ):
        ways.append(way)
allTypes.append(ways)

## 4 - [Monatsabhängige oder saisonale "Bitte um Nichtbefahrung" für Skifahrer](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngige_oder_saisonale_.22Bitte_um_Nichtbefahrung.22_f.C3.BCr_Skifahrer)

In [ ]:
ways = []
for way in result.ways:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(way.tags)

    if (
        (
            (
                not (seasonal is None or seasonal == 'no') 
            ) and (
                ski == 'discouraged'
                or re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
                or (
                    (
                        access == 'discouraged' 
                        or re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                    ) and (
                        ski is None
                        or not ski in ['yes','designated','permissive']
                    )
                )
            )
        ) or (
            re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', ski_conditional)
            or (
                (
                    re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_conditional)
                ) and (
                    ski is None
                )
            )        
        )
    ):
        ways.append(way)
allTypes.append(ways)

## 5 - [Ganzjähriges Skibefahrungsverbot auf der Fläche außerhalb der Wege (Wegegebot)](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hriges_Skibefahrungsverbot_auf_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege_.28Wegegebot.29)

In [ ]:
ways = []
for way in result.ways:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(way.tags)

    if ( 
        seasonal is None 
        or seasonal == 'no' 
    ) and (
        access_offroad in ['no','private'] and (
            ski is None
            or not ski in ['yes','designated','permissive']
        )
    ):
        ways.append(way)
allTypes.append(ways)

## 6 - [Monatsabhängiges oder saisonales Skibefahrungsverbot auf der Fläche außerhalb der Wege (saisonales Wegegebot)](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngiges_oder_saisonales_Skibefahrungsverbot_auf_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege_.28saisonales_Wegegebot.29)

In [ ]:
ways = []
for way in result.ways:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(way.tags)
    if (
        (
            (
                not (seasonal is None or seasonal == 'no')
            ) and (
                access_offroad in ['no','private']
                or re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
            ) and (
                ski is None
                or not ski in ('yes','designated','permissive')
            )
        )
    ) or (
        re.match('(no|private) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
        and (
            ski is None
        )
    ):
        ways.append(way)
allTypes.append(ways)

## 7 - [Ganzjährige "Bitte um Nichtbefahrung" der Fläche außerhalb der Wege](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Ganzj.C3.A4hrige_.22Bitte_um_Nichtbefahrung.22_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege)

In [ ]:
ways = []
for way in result.ways:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(way.tags)

    if ( 
        seasonal is None 
        or seasonal == 'no'
    ) and (
        access_offroad == 'discouraged' and (
            ski is None
            or not ski in ['yes','designated','permissive']
        )
    ):
        ways.append(way)
allTypes.append(ways)

## 8 - [Monatsabhängige oder saisonale "Bitte um Nichtbefahrung" auf der Fläche außerhalb der Wege](https://wiki.openstreetmap.org/wiki/DE:Betretungsverbote_f%C3%BCr_Gebiete_im_Winter#Monatsabh.C3.A4ngige_oder_saisonale_.22Bitte_um_Nichtbefahrung.22_auf_der_Fl.C3.A4che_au.C3.9Ferhalb_der_Wege)

In [ ]:
ways = []
for way in result.ways:
    seasonal, ski, ski_conditional, access, access_conditional, access_offroad, access_offroad_conditional = get_tags(way.tags)

    if (
        (
            (
                not (seasonal is None or seasonal == 'no')
            ) and (
                access_offroad == 'discouraged' 
                or re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
            ) and (
                ski is None
                or not ski in ['yes','designated','permissive']
            )
        )
    ) or (
        re.match('(discouraged) *@ *\(?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) *[0-9]* *[-,\,]? *(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)* *[0-9]*\)?', access_offroad_conditional)
        and (
            ski is None
        )
    ):
        ways.append(way)
allTypes.append(ways)

## Ergebnisse:

In [ ]:
if not silent:
    def href(id):
        return f"<a href='https://www.openstreetmap.org/way/{id}'>{id}</a>"

    subSum = 0
    for i in range(0, 7):
        subSum += len(allTypes[i])

    # if subSum != len(result.ways):
    #     print(f"Total no of ways={len(result.ways)}, sum of types={subSum}")

    multiClassifiedWays = []
    unclassifiedWays = []
    for way in result.ways:
        types = []
        for i in range(0, 7):
            if way in allTypes[i]:
                types.append(f"type{i+1}")
        way.tags['classification'] = types
        if len(types) == 0:
            unclassifiedWays.append(way)
        if len(types) > 1:
            multiClassifiedWays.append(way)

    html = f"<h3>Multiple Classifications [{len(multiClassifiedWays)}]</h3><ul>"
    for way in multiClassifiedWays:
        wayId = href(way.id)
        html += f"<li>{wayId} -> {way.tags['classification']}</li>"
    display(HTML(f"{html}</u>"))

    html = f"<h3>Unclassified [{len(unclassifiedWays)}]</h3><ul>"
    for way in unclassifiedWays:
        wayId = href(way.id)
        html += f"<li>{wayId} -> {way.tags}</li>"
    display(HTML(f"{html}</u>"))

## Create a geojson with explicit tags for the 8 types
* tag of the property is s_type, values are type1 ... type8
* styling is still work in progress, for now, there's one single style for all types
  * for http://geojson.io this is limited by what styling capabilities are available there
  * TBD for leaflet (and a "reference" map)

In [ ]:
from geojson import FeatureCollection, Feature, Polygon
features = []

styles = {"stroke": "#555555", "stroke-width": 2, "stroke-opacity": 1, "fill": "#ffff80", "fill-opacity": 0.4}

for i in range(0, 7):
    for way in allTypes[i]:
        lonLats = []
        for node in way.nodes:
            lonLats.append((float(node.lon),float(node.lat)))
        way.tags['s_type'] = f"type{i+1}"
        way.tags.update(styles)
        features.append(Feature(geometry=Polygon([lonLats]),properties=way.tags))

feature_collection = FeatureCollection(features)

if not os.path.isdir('data'):
    os.mkdir('data')

with open(f"data/Schongebiete.geojson", "w") as gFile:
    geojson.dump(feature_collection, gFile)
